In [53]:
# Master File for CMS Run 3 Level 1 Muon RoI Trigger Offline Studies

# Coders: Nathan Suri, Caltech; Cristian Pena, Caltech/Fermilab
# Date: July 2019
# LPC LLP Group

# Description
# Compiled progress on offline trigger study for CMS Run 3 Level 1 Muon RoI Trigger

# Action Plan
# Understand phi granularity implementation

# Notes/Conclusions

In [54]:
# Setups pwd location for data files and imports of special ROOT utilities

work_location = input("Username: ")
if work_location == 'nasurijr':
    pwd = '/nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/'
# elif work_location == '<Insert Tier2 username here>':
#     pwd = '/home/cms/delayed_jet_analyzer/'

Username: nasurijr


# Imports

In [55]:
# Imports necessary utilities and modules

import ROOT as rt
import root_numpy as rtnp
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from collections import Counter 
import datetime
import pytz

# Graph/histo utilities from ROOT
# Contained within the delayed_jet_analyzer repository
import sys
sys.path.append(pwd+'lib')
from histo_utilities import create_TH1D, create_TH2D, create_TGraph, std_color_list

# Used for extracting the TTree structure from each datafile
import os
import uproot

# Sets display width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# Imports jet clustering algorithm (FastJet)
from pyjet import cluster


donotdelete = []

# Read in and Extract TTrees from Datafiles

In [56]:
# Setups dictionaries for storing data from MC/data ntuples
fpath = {}
tree = {}

data_path = pwd+'data/'

# Background Samples
fpath['qcd'] = data_path +'jet_timing_studies_ntuple_RunIIFall17DRPremix_QCD_Pt_170to300_TuneCP5_13TeV_pythia8_1.root'

# Small subset of B samples for quick runs/tests
fpath['zeroBias'] = data_path + 'jet_timing_studies_ZeroBias_Run2018B_112_dec.root'
# # Complete set of A and B ZeroBias samples (time-intensive to run)
# fpath['zeroBias'] = data_path + 'jet_timing_studies_ZeroBias_Run2018AB_complete.root'

# WH-comparable background: WJetsToLNu
fpath['WJetsToLNu'] = data_path + 'jet_timing_studies_ntuple_RunIIFall17DRPremix_WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8.root'

################################################################

# Signal Samples
# mH = 125 GeV, mX = 50 GeV, ctau = 1 m
fpath['m50ct1000mm'] = data_path+'jet_timing_studies_ntuple_metbb_ggh_ISR_mh125_mx50_pl1000_ev100000.root'

# mH = 125 GeV, mX = 50 GeV, ctau = 10 m
fpath['m50ct10000mm'] = data_path+'jet_timing_studies_ntuple_metbb_ggh_ISR_mh125_mx50_pl10000_ev100000.root'

# mH = 2000 GeV, mX = 975 GeV, ctau = 1 m
fpath['m975ct1000mm'] = data_path+'jet_timing_studies_ntuple_metbb_ggh_ISR_mh2000_mx975_pl1000_ev100000.root'

# mH = 2000 GeV, mX = 975 GeV, ctau = 10 m
fpath['m975ct10000mm'] = data_path+'jet_timing_studies_ntuple_metbb_ggh_ISR_mh2000_mx975_pl10000_ev100000.root'

fpath['l1_nik_m50ct1000mm'] = data_path+'EMTF_ppTohToSS1SS2_SS1Tobb_SS2Tobb_ggh_withISR.root'

# Iterates through each file and extracts the ROOT TTree structure from each
for k,v in fpath.items():
    print(str(datetime.datetime.now(pytz.timezone('US/Pacific'))))
    print(k, v)
    root_dir = uproot.open(v)
    if k == 'l1_nik_m50ct1000mm':
        tree[k] = root_dir['FlatNtupleMC']['tree']
    else:
        tree[k] = root_dir['ntuples']['llp']
#     print(root_dir['ntuples']['NEvents'][1])
    # Accesses the array form of the jetPt branch
#     a = tree[k]["jetPt"].array()
#     print(a[:-1][0])

2019-08-30 13:57:58.958241-07:00
qcd /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/jet_timing_studies_ntuple_RunIIFall17DRPremix_QCD_Pt_170to300_TuneCP5_13TeV_pythia8_1.root
2019-08-30 13:57:59.619567-07:00
zeroBias /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/jet_timing_studies_ZeroBias_Run2018B_112_dec.root
2019-08-30 13:58:00.287680-07:00
WJetsToLNu /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/jet_timing_studies_ntuple_RunIIFall17DRPremix_WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8.root
2019-08-30 13:58:00.783152-07:00
m50ct1000mm /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/jet_timing_studies_ntuple_metbb_ggh_ISR_mh125_mx50_pl1000_ev100000.root
2019-08-30 13:58:01.321517-07:00
m50ct10000mm /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/jet_timing_studies_ntuple_metbb_ggh_ISR_mh125_mx50_pl10000_ev100000.root
2019-08-30 13:58:01.728785-07:00
m975ct1000mm /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/jet_timing_studi

## Name TTree Objects

In [57]:
# Bookkeeping: Defines the TTrees from the read datafiles
# Names displayed in README.md table

T = tree['m50ct1000mm']
T_bkg = tree['qcd']
T_minBias = tree['zeroBias']
T_low_ctau10 = tree['m50ct10000mm']
T_high_ctau1 = tree['m975ct1000mm']
T_high_ctau10 = tree['m975ct10000mm']
T_l1 = tree['l1_nik_m50ct1000mm']

# Define Variables and Name Datafiles 

In [58]:
# Variable Definitions
# Creates dictionaries for variables to be analyzed
# The dictionaries will contain the variable arrays for each datafile with a relevant key

# CSC
nCsc = {}
csc_z = {}
csc_x = {}
csc_y = {}
csc_eta = {}
csc_phi = {}
# Gen Level
gLLP_eta = {}
gLLP_r = {}
gLLP_decay = {}
eventNum_endcap = {}
lumiNum_endcap = {}
eventNum_barrel = {}
lumiNum_barrel = {}
eventNum_bkg = {}
lumiNum_bkg = {}
ncsc_barrel = {}
gLLPr = {}

# Drift Tube (DT)
nDt = {}
dt_phi = {}
dt_eta = {}
dt_z = {}
dt_x = {}
dt_y = {}
# RPC
nRpc = {}
rpc_phi = {}
rpc_eta = {}
rpc_z = {}
rpc_x = {}
rpc_y = {}

In [59]:
# Bookkeeping: Creates a dictionary for iterating over all of the datafiles and 
#              converting the relevant branches to numpy arrays
# Names displayed in README.md table

data_trees = {'m50ct1m': T, 'qcd': T_bkg, 'zeroBias':T_minBias, 'm50ct10m': T_low_ctau10, 'm975ct1m': T_high_ctau1, 'm975ct10m': T_high_ctau10}

# Event Selection

## TBranch -> np.array() Conversion

In [60]:
# Event Selection

print('Start: ' + str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

# MET+bb Final State
if 'metbb' in fpath['m50ct1000mm']:
    for species, arbor in data_trees.items():
        
        gLLP_r[species] = np.sqrt((arbor['gLLP_decay_vertex_x'].array()[:,0])**2+(arbor['gLLP_decay_vertex_y'].array()[:,0])**2)
        gLLP_decay[species] = np.sqrt((arbor['gLLP_decay_vertex_x'].array()[:,0])**2+(arbor['gLLP_decay_vertex_y'].array()[:,0])**2 + (arbor['gLLP_decay_vertex_z'].array()[:,0])**2)
        
        # Endcap Signal Event Selection: Requires that the bb-decaying LLP decays within the endcap of the muon system        
        sel_csc = np.logical_and(np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) > 568 , np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) < 1100)
        sel_csc = np.logical_and(sel_csc, np.absolute(arbor['gLLP_eta'].array()[:,0])<2.4)
        sel_csc = np.logical_and(sel_csc, np.absolute(arbor['gLLP_eta'].array()[:,0])>0.9)
        sel_csc = np.logical_and(sel_csc, np.absolute(gLLP_r[species])<695.5)

        # Barrel Signal Event Selection: Requires that the bb-decaying LLP decays within the barrel of the muon system        
        sel_barrel = np.logical_and(np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) > -661 , np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) < 661)
        sel_barrel = np.logical_and(sel_barrel, np.absolute(gLLP_r[species])<738)
        sel_barrel = np.logical_and(sel_barrel, np.absolute(gLLP_r[species])>380)

        # Overlap Signal Event Selection: Requires that the bb-decaying LLP decays within the overlap of the barrel
        #                                 and endcap of the muon system
        sel_rpc = np.logical_and(np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) > 0 , np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) < 1100)
        sel_rpc = np.logical_and(sel_rpc, np.absolute(arbor['gLLP_eta'].array()[:,0])<1.6)
        sel_rpc = np.logical_and(sel_rpc, np.absolute(gLLP_r[species])<738)
        sel_rpc = np.logical_and(sel_rpc, np.absolute(gLLP_r[species])>275)

        # Total Muon System Signal Event Selection: Logical or of barrel and endcap gen-level restrictions
        sel_dis_signal = np.logical_or(sel_csc, sel_barrel)

        if 'vh' in fpath['m50ct1000mm']:
            # Basic implementation: Loop through files and compile Boolean array with 1 muon/electron
            mu = np.logical_and(np.logical_and(np.absolute(gParticleId[ev]) == 13, np.absolute(gParticleMotherId[ev]) == 24), gParticleStatus[ev] == 1 )
            ele = np.logical_and(np.logical_and(np.absolute(gParticleId[ev]) == 11, np.absolute(gParticleMotherId[ev]) == 24), gParticleStatus[ev] == 1 )
            if not ((np.count_nonzero(mu) ==1 or np.count_nonzero(ele)==1)): continue
        
        # Converts variable branches to numpy arrays with event selection
        # Signal conversions
        if species in ('m50ct1m', 'm50ct10m', 'm975ct1m', 'm975ct10m'):
            # Endcap: CSCs
            nCsc[species] = arbor['nCsc'].array()[sel_csc]
            csc_z[species] = arbor['cscZ'].array()[sel_csc]
            csc_x[species] = arbor['cscX'].array()[sel_csc]
            csc_y[species] = arbor['cscY'].array()[sel_csc]
            csc_eta[species] = arbor['cscEta'].array()[sel_csc]
            csc_phi[species] = arbor['cscPhi'].array()[sel_csc]
            
            # Gen-level
            gLLP_eta[species] = arbor['gLLP_eta'].array()[:,0][sel_csc]
            eventNum_endcap[species] = arbor['eventNum'].array()[sel_csc]
            lumiNum_endcap[species] = arbor['lumiNum'].array()[sel_csc]
            
            eventNum_barrel[species] = arbor['eventNum'].array()[sel_barrel]
            lumiNum_barrel[species] = arbor['lumiNum'].array()[sel_barrel]
            gLLPr[species] = gLLP_r[species][sel_barrel]
            ncsc_barrel = arbor['nCsc'].array()[sel_barrel]
            
            # Barrel: DTs
            nDt[species] = arbor['nDt'].array()[sel_barrel]
            dt_z[species] = arbor['dtZ'].array()[sel_barrel]
            dt_x[species] = arbor['dtX'].array()[sel_barrel]
            dt_y[species] = arbor['dtY'].array()[sel_barrel]
            dt_eta[species] = arbor['dtEta'].array()[sel_barrel]
            dt_phi[species] = arbor['dtPhi'].array()[sel_barrel]
            
            # Overlap: RPCs
            nRpc[species] = arbor['nRpc'].array()[sel_rpc]
            rpc_z[species] = arbor['rpcZ'].array()[sel_rpc]
            rpc_x[species] = arbor['rpcX'].array()[sel_rpc]
            rpc_y[species] = arbor['rpcY'].array()[sel_rpc]
            rpc_eta[species] = arbor['rpcEta'].array()[sel_rpc]
            rpc_phi[species] = arbor['rpcPhi'].array()[sel_rpc]
        
        # Background conversions
        if species in ('qcd', 'zeroBias'):
            sel_bkg = np.ones(len(arbor['nCsc'].array()), dtype=bool)
            
            # Endcap: CSCs
            nCsc[species] = arbor['nCsc'].array()[sel_bkg]
            csc_z[species] = arbor['cscZ'].array()[sel_bkg]
            csc_x[species] = arbor['cscX'].array()[sel_bkg]
            csc_y[species] = arbor['cscY'].array()[sel_bkg]
            csc_eta[species] = arbor['cscEta'].array()[sel_bkg]
            csc_phi[species] = arbor['cscPhi'].array()[sel_bkg]
            gLLP_eta[species] = arbor['gLLP_eta'].array()[:,0][sel_bkg]
            
            # Gen-level
            eventNum_bkg[species] = arbor['eventNum'].array()[sel_bkg]
            lumiNum_bkg[species] = arbor['lumiNum'].array()[sel_bkg]
            
            # Barrel: DTs
            nDt[species] = arbor['nDt'].array()[sel_bkg]
            dt_z[species] = arbor['dtZ'].array()[sel_bkg]
            dt_x[species] = arbor['dtX'].array()[sel_bkg]
            dt_y[species] = arbor['dtY'].array()[sel_bkg]
            dt_eta[species] = arbor['dtEta'].array()[sel_bkg]
            dt_phi[species] = arbor['dtPhi'].array()[sel_bkg]
            
            # Overlap: RPCs
            nRpc[species] = arbor['nRpc'].array()[sel_bkg]
            rpc_z[species] = arbor['rpcZ'].array()[sel_bkg]
            rpc_x[species] = arbor['rpcX'].array()[sel_bkg]
            rpc_y[species] = arbor['rpcY'].array()[sel_bkg]
            rpc_eta[species] = arbor['rpcEta'].array()[sel_bkg]
            rpc_phi[species] = arbor['rpcPhi'].array()[sel_bkg]
        
        print(species + ': ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

Start: 2019-08-30 13:58:03.575140-07:00
m50ct1m: 2019-08-30 13:58:04.629534-07:00
qcd: 2019-08-30 13:58:05.896163-07:00


KeyboardInterrupt: 

## L1 Trigger Primitives

In [ ]:
nCsc['l1_m50ct1m'] = T_l1['nHits'].array()

## Second Moment Calculation

In [ ]:
fake_pt_csc = {}
fake_mass_csc = {}

print('Start: ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

for species in data_trees.keys():
    event_empty = np.copy(csc_z[species])*(-999)
    fake_pt_csc[species] = event_empty
    fake_mass_csc[species] = event_empty
    
    print(species + ': ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

In [ ]:
cluster_dataset = {}

print('Start: ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

for species in data_trees.keys():
    cluster_dataset[species] = np.dstack((fake_pt_csc[species], csc_eta[species], csc_phi[species], fake_mass_csc[species], csc_z[species]))

    print(species + ': ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

In [ ]:
csc_clusters = {}
ncsc_cluster = {}
eta_max_cluster = {}

print('Start: ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

for species in data_trees.keys():
    csc_clusters[species] = []
    ncsc_cluster[species] = []
    eta_max_cluster[species] = []
    for hit_seq in range(len(csc_z[species])):
#         event_clusters = []
        test_data_0 = np.dstack(cluster_dataset[species][0][hit_seq])[0].astype(np.float64)
        test_data_0 = np.core.records.fromarrays(test_data_0.transpose(), names='pT, eta, phi, mass, z_pos', formats = 'f8, f8, f8, f8, f8')
        sequence_0 = cluster(test_data_0, R=0.4, p=0)
        jets = sequence_0.inclusive_jets()
#         event_clusters.append(jets)
#         csc_clusters[species].append(event_clusters)
        csc_clusters[species].append(jets)

        total_hits = 0
        for cluster_hits in range(len(jets)):
            if len(jets[cluster_hits].constituents_array()) > 3:
                total_hits += len(jets[cluster_hits].constituents_array())
        ncsc_cluster[species].append(total_hits)
        
        if len(jets) == 0:
            continue
        max_pop_index = np.where(np.max(len(jets)))
        
        eta_max_cluster[species].append(jets[max_pop_index[0][0]].constituents_array()['eta'])
    
    print(species + ': '+  str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

In [ ]:
def moment_phi(phi_array):
    moment_2 = []
    for event in phi_array:
        avg_phi = np.mean(event)
        moment_array = []
        for segment in event:
            moment = (segment-avg_phi)**2
            moment_array.append(moment)
        moment_2.append(np.sum(moment_array))
    return moment_2

In [ ]:
moment_2_phi = {}
for species in data_trees.keys():
    moment_2_phi[species] = []
    moment_2_phi[species].append(moment_phi(eta_max_cluster[species]))

In [ ]:
# Visualizes phi spread of most populated cluster

c_phi = rt.TCanvas('c_phi','c_phi', 800, 600)

h={}
h['signal_phi'] = create_TH1D(moment_2_phi['m50ct1m'][0], axis_title=['signal_phi', 'Events'], name='signal_phi', binning=[100,0,6.3])
h['signal_phi'].SetLineColor(4)


h['bkg_phi'] = create_TH1D(moment_2_phi['zeroBias'][0], axis_title=['bkg_phi', 'Events'], name='bkg_phi', binning=[100,0,6.3])
h['bkg_phi'].SetLineColor(2)

c_phi.SetLogy()

h['signal_phi'].SetLineWidth(2)
h['bkg_phi'].SetLineWidth(2)

# h['bkg_phi'].GetYaxis().SetRangeUser(1,10**10)
h['bkg_phi'].GetXaxis().SetRangeUser(0,6.3)


h['bkg_phi'].SetStats(0)
h['bkg_phi'].SetTitle("CSC Inclusive (ggH #rightarrow MET + bb)")
h['bkg_phi'].SetXTitle("m_{#phi}")


h['bkg_phi'].Draw('histo')
h['signal_phi'].Draw('histo+same')


legend = rt.TLegend(0.48,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
legend.AddEntry( h['signal_phi'], "ggH Second Moment" , "L");
legend.AddEntry( h['bkg_phi'], "zeroBias Second Moment" , "L");

legend.Draw();

c_phi.Draw()

# c.SaveAs("ncsc_inclusive_mc_zeroBias.pdf")
# c.SaveAs("ncsc_inclusive_mc_zeroBias.C")

## Endcap Hit Removal

In [ ]:
# Hit Removal
# For both the endcap and barrel regions (overlap to be determined), the closest stations to the beamspot are the noisiest.
# Thus, we remove the hits that land in these stations for each event in each datafile.

# Endcap
nCsc_station = {}
nCsc_removal = {}

# Loops over every datafile
for files in data_trees.keys():
    # Endcap
    nCsc_station[files] = {}
    nCsc_station[files]['ME_11'] = []
    nCsc_station[files]['ME_12'] = []
    nCsc_station[files]['ME_13'] = []
    
    # Loops over every event in each datafile
    for pos_bool in range(len(csc_z[files])):
        csc_r = np.sqrt(csc_x[files][pos_bool]**2+csc_y[files][pos_bool]**2)
        z_pos_bool = csc_z[files][pos_bool]
        
        counter_me11 = 0
        counter_me12 = 0
        counter_me13 = 0
        
        # Loops over every hit in each event
        for z_ind in range(len(z_pos_bool)):
            z_bool = np.absolute(z_pos_bool[z_ind])
            # ME 1/1 Constraints
            if z_bool > 568 and z_bool < 632:
                counter_me11 += 1
                
            if z_bool > 663 and z_bool < 724:
                # ME 1/2 Constraints
                if csc_r[z_ind] < 465 and csc_r[z_ind] > 275:
                    counter_me12 += 1
                # ME 1/3 Constraints
                if csc_r[z_ind] < 695.5 and csc_r[z_ind] > 505.5:
                    counter_me13 += 1
                    
        nCsc_station[files]['ME_11'].append(counter_me11)
        nCsc_station[files]['ME_12'].append(counter_me12)
        nCsc_station[files]['ME_13'].append(counter_me13)
    
    # nCsc dictionaries containing hits removed labelled by removed stations
    nCsc_removal[files] = {}
    nCsc_removal[files]['ME_11'] = np.array(nCsc[files]) - np.array(nCsc_station[files]['ME_11'])
    nCsc_removal[files]['ME_112'] = np.array(nCsc[files]) - (np.array(nCsc_station[files]['ME_11']) + np.array(nCsc_station[files]['ME_12']))
    nCsc_removal[files]['ME_1123'] = np.array(nCsc[files]) - (np.array(nCsc_station[files]['ME_11']) + np.array(nCsc_station[files]['ME_12']) + np.array(nCsc_station[files]['ME_13']))
    
    print(files + ': ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

In [ ]:
print(len(nCsc['m50ct10m']))

In [ ]:
# # Hit Selection based on Phi Granularity

pg_ncsc = {}
pg_ncsc_noME11 = {}
pg_ncsc_noME112 = {}
pg_ncsc_noME1123 = {}

print('Start: ' + str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

for species in data_trees.keys():
    pg_ncsc[species] = []
    pg_ncsc_noME11[species] = []
    pg_ncsc_noME112[species] = []
    pg_ncsc_noME1123[species] = []
    
    for event in range(len(csc_z[species])):
        cscR = np.sqrt(csc_x[species][event]**2+csc_y[species][event]**2)
        cscZ = csc_z[species][event]
        cscPhi = csc_phi[species][event] * (180/np.pi)
        
        pg_me11 = {'sectors':np.zeros(36), 'counter':0}
        pg_me12 = {'sectors':np.zeros(36), 'counter':0}
        pg_me13 = {'sectors':np.zeros(36), 'counter':0}
        pg_me21 = {'sectors':np.zeros(36), 'counter':0}
        pg_me22 = {'sectors':np.zeros(36), 'counter':0}
        pg_me31 = {'sectors':np.zeros(36), 'counter':0}
        pg_me32 = {'sectors':np.zeros(36), 'counter':0}
        pg_me41 = {'sectors':np.zeros(36), 'counter':0}
        pg_me42 = {'sectors':np.zeros(36), 'counter':0}
        
        
        for zIndex in range(len(cscZ)):
            hit_z = np.absolute(cscZ[zIndex])
            hit_phi = cscPhi[zIndex]
            hit_r = cscR[zIndex]
            
            # ME 1/1
            if hit_z > 568 and hit_z < 632:
                index = int((hit_phi+180)/10.0)
                pg_me11['sectors'][index - 1] += 1

#             # ME 1/2 and ME 1/3
            if hit_z > 663 and hit_z < 724:
                if hit_r < 465 and hit_r > 275:
                    index = int((hit_phi+180)/10.0)
                    pg_me12['sectors'][index - 1] += 1
                if hit_r < 695.5 and hit_r > 505.5:
                    index = int((hit_phi+180)/10.0)
                    pg_me13['sectors'][index - 1] += 1
    
            # ME 2/1 and ME 2/2
            if hit_z > 791 and hit_z < 849.5:
                if hit_r < 345 and hit_r > 138.5:
                    index = int((hit_phi+180)/20.0)
                    pg_me21['sectors'][index - 1] += 1
                if hit_r < 695.5 and hit_r > 357.5:
                    index = int((hit_phi+180)/10.0)
                    pg_me22['sectors'][index - 1] += 1

            # ME 3/1 and ME 3/2
            if hit_z > 911.5 and hit_z < 970:
                if hit_r < 345 and hit_r > 160.5:
                    index = int((hit_phi+180)/20.0)
                    pg_me31['sectors'][index - 1] += 1
                if hit_r < 695.5 and hit_r > 357.5:
                    index = int((hit_phi+180)/10.0)
                    pg_me32['sectors'][index - 1] += 1
                
            # ME 4/1 and ME 4/2
            if hit_z > 1002 and hit_z < 1060.5:
                if hit_r < 345 and hit_r > 177.5:
                    index = int((hit_phi+180)/20.0)
                    pg_me41['sectors'][index - 1] += 1
                if hit_r < 695.5 and hit_r > 357.5:
                    index = int((hit_phi+180)/10.0)
                    pg_me42['sectors'][index - 1] += 1
                    
        pg_me11['counter'] += np.count_nonzero(pg_me11['sectors'])
        pg_me12['counter'] += np.count_nonzero(pg_me12['sectors'])
        pg_me13['counter'] += np.count_nonzero(pg_me13['sectors'])
        pg_me21['counter'] += np.count_nonzero(pg_me21['sectors'])
        pg_me22['counter'] += np.count_nonzero(pg_me22['sectors'])
        pg_me31['counter'] += np.count_nonzero(pg_me31['sectors'])
        pg_me32['counter'] += np.count_nonzero(pg_me32['sectors'])
        pg_me41['counter'] += np.count_nonzero(pg_me41['sectors'])
        pg_me42['counter'] += np.count_nonzero(pg_me42['sectors'])
        
#         print(pg_me11['sectors'])
#         print(np.count_nonzero(pg_me11['sectors']))
#         print(np.where(pg_me11['sectors'] >= 2)[0].size)
        
#         break

        pg_me11['counter'] += np.where(pg_me11['sectors'] >= 2)[0].size
        pg_me12['counter'] += np.where(pg_me12['sectors'] >= 2)[0].size
        pg_me13['counter'] += np.where(pg_me13['sectors'] >= 2)[0].size
        pg_me21['counter'] += np.where(pg_me21['sectors'] >= 2)[0].size
        pg_me22['counter'] += np.where(pg_me22['sectors'] >= 2)[0].size
        pg_me31['counter'] += np.where(pg_me31['sectors'] >= 2)[0].size
        pg_me32['counter'] += np.where(pg_me32['sectors'] >= 2)[0].size
        pg_me41['counter'] += np.where(pg_me41['sectors'] >= 2)[0].size
        pg_me42['counter'] += np.where(pg_me42['sectors'] >= 2)[0].size
        

        cscSum_inclusive = pg_me11['counter'] + pg_me12['counter'] + pg_me13['counter'] + pg_me21['counter'] + pg_me22['counter'] + pg_me31['counter'] + pg_me32['counter'] + pg_me41['counter'] + pg_me42['counter']
        cscSum_noME11 = pg_me12['counter'] + pg_me13['counter'] + pg_me21['counter'] + pg_me22['counter'] + pg_me31['counter'] + pg_me32['counter'] + pg_me41['counter'] + pg_me42['counter']
        cscSum_noME112 = pg_me13['counter'] + pg_me21['counter'] + pg_me22['counter'] + pg_me31['counter'] + pg_me32['counter'] + pg_me41['counter'] + pg_me42['counter']
        cscSum_noME1123 = pg_me21['counter'] + pg_me22['counter'] + pg_me31['counter'] + pg_me32['counter'] + pg_me41['counter'] + pg_me42['counter']
        
        pg_ncsc[species].append(cscSum_inclusive)
        pg_ncsc_noME11[species].append(cscSum_noME11)
        pg_ncsc_noME112[species].append(cscSum_noME112)
        pg_ncsc_noME1123[species].append(cscSum_noME1123)
        
    print(species + ': ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

In [ ]:
# Compares L1 trigger primitives to translated reco variable

c_pg = rt.TCanvas('c_pg','c_pg', 800, 600)

h['reco_inc'] = create_TH1D(pg_ncsc['m50ct1m'], axis_title=['reco_inc', 'Events'], name='reco_inc', binning=[100,0,200])
h['reco_inc'].SetLineColor(4)
h['reco_inc'].SetLineStyle(1)

h['reco_bkg'] = create_TH1D(pg_ncsc['qcd'], axis_title=['reco_bkg', 'Events'], name='reco_bkg', binning=[100,0,200])
h['reco_bkg'].SetLineColor(2)
h['reco_bkg'].SetLineStyle(1)


h['reco_no11'] = create_TH1D(pg_ncsc_noME11['m50ct1m'], axis_title=['reco_no11', 'Events'], name='reco_no11', binning=[100,0,200])
h['reco_no11'].SetLineColor(1)
h['reco_no11'].SetLineStyle(1)

h['reco_no112'] = create_TH1D(pg_ncsc_noME112['m50ct1m'], axis_title=['reco_no112', 'Events'], name='reco_no112', binning=[100,0,200])
h['reco_no112'].SetLineColor(6)
h['reco_no112'].SetLineStyle(1)

h['reco_no1123'] = create_TH1D(pg_ncsc_noME1123['m50ct1m'], axis_title=['reco_no1123', 'Events'], name='reco_no1123', binning=[100,0,200])
h['reco_no1123'].SetLineColor(8)
h['reco_no1123'].SetLineStyle(1)


h['l1_inc'] = create_TH1D(nCsc['l1_m50ct1m'], axis_title=['l1_inc', 'Events'], name='l1_inc', binning=[100,0,200])
h['l1_inc'].SetLineColor(4)
h['l1_inc'].SetLineStyle(2)

c_pg.SetLogy()

h['reco_inc'].SetLineWidth(2)
h['reco_no11'].SetLineWidth(2)
h['reco_no112'].SetLineWidth(2)
h['reco_no1123'].SetLineWidth(2)
h['l1_inc'].SetLineWidth(2)
h['reco_bkg'].SetLineWidth(2)

h['reco_inc'].GetYaxis().SetRangeUser(1,10**5)
h['reco_inc'].GetXaxis().SetRangeUser(0,55)


h['reco_inc'].SetStats(0)
h['reco_inc'].SetTitle("Phi Granularity (ggH #rightarrow MET + bb)")
h['reco_inc'].SetXTitle("N_{CSC/LCT}")


h['reco_inc'].Draw('histo')
h['reco_no11'].Draw('histo+same')
h['reco_no112'].Draw('histo+same')
h['reco_no1123'].Draw('histo+same')
h['l1_inc'].Draw('histo+same')
h['reco_bkg'].Draw('histo+same')



# legend = rt.TLegend(0.48,0.70,0.87,0.87);
legend = rt.TLegend(0.6,0.60,0.6,0.6);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
legend.AddEntry( h['reco_inc'], "ggH Inclusive" , "L");
legend.AddEntry( h['reco_no11'], "ggH w/o ME 11" , "L");
legend.AddEntry( h['reco_no112'], "ggH w/o ME 11+12" , "L");
legend.AddEntry( h['reco_no1123'], "ggH w/o ME 11+12+13" , "L");
legend.AddEntry( h['reco_bkg'], "QCD Inclusive" , "L");
legend.AddEntry( h['l1_inc'], "ggH LCT Inclusive" , "L");

legend.Draw();

c_pg.Draw()

# c.SaveAs("ncsc_inclusive_mc_zeroBias.pdf")
# c.SaveAs("ncsc_inclusive_mc_zeroBias.C")

## Barrel Hit Removal

In [ ]:
# Hit removal done identically to that of the endcap

# Barrel
nDt_station = {}
nDt_removal = {}

# Loops over every datafile
for files in data_trees.keys():
    # Barrel
    nDt_station[files] = {}
    nDt_station[files]['MB_1'] = []
    nDt_station[files]['MB_2'] = []
    nDt_station[files]['MB_3'] = []
    nDt_station[files]['MB_4'] = []
    
    # Loops over every event in each datafile
    for pos_bool in range(len(dt_z[files])):
        dt_r = np.sqrt(dt_x[files][pos_bool]**2+dt_y[files][pos_bool]**2)
        z_pos_bool = dt_z[files][pos_bool]
        
        counter_mb1 = 0
        counter_mb2 = 0
        counter_mb3 = 0
        counter_mb4 = 0
        
        # Loops over every hit in each event
        for z_ind in range(len(z_pos_bool)):
            z_bool = np.absolute(z_pos_bool[z_ind])
            if z_bool > -661 and z_bool < 661:
                
                # MB 1 Constraints
                if dt_r[z_ind] < 449 and dt_r[z_ind] > 402:
                    counter_mb1 += 1
                # MB 2 Constraints
                if dt_r[z_ind] < 533.5 and dt_r[z_ind] > 490.5:
                    counter_mb2 += 1
                # MB 3 Constraints
                if dt_r[z_ind] < 636 and dt_r[z_ind] > 597.5:
                    counter_mb3 += 1
                # MB 4 Constraints
                if dt_r[z_ind] < 738 and dt_r[z_ind] > 700:
                    counter_mb4 += 1
                    
        nDt_station[files]['MB_1'].append(counter_mb1)
        nDt_station[files]['MB_2'].append(counter_mb2)
        nDt_station[files]['MB_3'].append(counter_mb3)
        nDt_station[files]['MB_4'].append(counter_mb4)    
    
    # nDt dictionaries containing hits removed labelled by removed stations
    nDt_removal[files] = {}
    nDt_removal[files]['MB_1'] = np.array(nDt[files]) - np.array(nDt_station[files]['MB_1'])
    nDt_removal[files]['MB_12'] = np.array(nDt[files]) - (np.array(nDt_station[files]['MB_1']) + np.array(nDt_station[files]['MB_2']))
    
    print(files + ': ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

## Calculation of Average Fraction of Hits per DT station

In [ ]:
# # Determines the evenness of hits distributed across the barrel DT stations by taking the average ratio of nDt for adjacent stations per event

# nDt_avg_ratio = {}
# nDt_ratio_cut = {}
    
# for species in data_trees.keys():
#     nDt_ratio_cut[species] = {}
#     ratio_mb_12 = []
#     ratio_mb_23 = []
#     ratio_mb_34 = []
#     for event in range(len(nDt_station[species]['MB_1'])):
#         mb_12_comp = np.min((nDt_station[species]['MB_1'][event], nDt_station[species]['MB_2'][event]))/np.max((nDt_station[species]['MB_1'][event], nDt_station[species]['MB_2'][event]))
#         ratio_mb_12.append(mb_12_comp)
        
#         mb_23_comp = np.min((nDt_station[species]['MB_2'][event], nDt_station[species]['MB_3'][event]))/np.max((nDt_station[species]['MB_2'][event], nDt_station[species]['MB_3'][event]))
#         ratio_mb_23.append(mb_23_comp)
        
#         mb_34_comp = np.min((nDt_station[species]['MB_3'][event], nDt_station[species]['MB_4'][event]))/np.max((nDt_station[species]['MB_3'][event], nDt_station[species]['MB_4'][event]))
#         ratio_mb_34.append(mb_34_comp)

#     barrel_station_ratio = np.dstack((np.array(ratio_mb_12), np.array(ratio_mb_23), np.array(ratio_mb_34)))
#     barrel_avg_ratio = np.mean(barrel_station_ratio, axis=2)
#     nDt_avg_ratio[species] = barrel_avg_ratio
#     sel_avg_ratio = nDt_avg_ratio[species][0]<7
#     nDt_ratio_cut[species]['dt_inclusive'] = nDt[species][sel_avg_ratio]
#     nDt_ratio_cut[species]['no_MB1'] = nDt_removal[species]['MB_1'][sel_avg_ratio]

## Calculations for N<sub>CSC, DT, RPC</sub> Threshold ROC Curves

In [ ]:
# Calculations for ROC Curves for ggH/QCD/ZeroBias Samples: nCsc/nDt/nRpc 

# Calculates efficiency by calculating the fraction of events that pass nCsc/nDt/nRpc thresholds

eff_ncsc = {}
eff_ndt = {}
eff_nrpc = {}
rejection_power = {}
ncsc_var_x = np.array([])
# ndt_var_x = np.array([])

# Iterates over every datafile
for data_type in data_trees.keys():
    eff_ncsc[data_type] = {}
    eff_ndt[data_type] = {}
    eff_nrpc[data_type] = {}
      
    # No clustering: Endcap
    eff_ncsc[data_type]['noCluster'] = {}
    eff_ncsc[data_type]['noCluster']['csc_inclusive'] = np.array([])
    eff_ncsc[data_type]['noCluster']['csc_noME11'] = np.array([])
    eff_ncsc[data_type]['noCluster']['csc_noME112'] = np.array([])
    eff_ncsc[data_type]['noCluster']['csc_noME1123'] = np.array([])
    eff_ncsc[data_type]['noCluster']['pg_inclusive'] = np.array([])
    eff_ncsc[data_type]['noCluster']['pg_noME11'] = np.array([])
    eff_ncsc[data_type]['noCluster']['pg_noME112'] = np.array([])
    eff_ncsc[data_type]['noCluster']['pg_noME1123'] = np.array([])
    
    # No clustering: Barrel
    eff_ndt[data_type]['noCluster'] = {}
    eff_ndt[data_type]['noCluster_ratioCut'] = {}
    
    eff_ndt[data_type]['noCluster']['dt_inclusive'] = np.array([])
    eff_ndt[data_type]['noCluster']['dt_noMB1'] = np.array([])
    eff_ndt[data_type]['noCluster']['dt_noMB12'] = np.array([])
    eff_ndt[data_type]['noCluster_ratioCut']['dt_inclusive'] = np.array([])
    eff_ndt[data_type]['noCluster_ratioCut']['dt_noMB1'] = np.array([])
        
    # No clustering: Overlap
    eff_nrpc[data_type]['noCluster'] = {}
    eff_nrpc[data_type]['noCluster']['rpc_inclusive'] = np.array([])
    
    for hit_thresh in range(0, 200):
        ncsc_var_x = np.append(ncsc_var_x, hit_thresh)
        
        # No-Clustering ROCs
        # Endcap
        eff_ncsc[data_type]['noCluster']['csc_inclusive'] = np.append(eff_ncsc[data_type]['noCluster']['csc_inclusive'], np.count_nonzero(np.array(nCsc[data_type])>hit_thresh)/len(nCsc[data_type]))
        eff_ncsc[data_type]['noCluster']['csc_noME11'] = np.append(eff_ncsc[data_type]['noCluster']['csc_noME11'], np.count_nonzero(np.array(nCsc_removal[data_type]['ME_11'])>hit_thresh)/len(nCsc_removal[data_type]['ME_11']))
        eff_ncsc[data_type]['noCluster']['csc_noME112'] = np.append(eff_ncsc[data_type]['noCluster']['csc_noME112'], np.count_nonzero(np.array(nCsc_removal[data_type]['ME_112'])>hit_thresh)/len(nCsc_removal[data_type]['ME_112']))
        eff_ncsc[data_type]['noCluster']['csc_noME1123'] = np.append(eff_ncsc[data_type]['noCluster']['csc_noME1123'], np.count_nonzero(np.array(nCsc_removal[data_type]['ME_1123'])>hit_thresh)/len(nCsc_removal[data_type]['ME_1123']))

        eff_ncsc[data_type]['noCluster']['pg_inclusive'] = np.append(eff_ncsc[data_type]['noCluster']['pg_inclusive'], np.count_nonzero(np.array(pg_ncsc[data_type])>hit_thresh)/len(pg_ncsc[data_type]))
        eff_ncsc[data_type]['noCluster']['pg_noME11'] = np.append(eff_ncsc[data_type]['noCluster']['pg_noME11'], np.count_nonzero(np.array(pg_ncsc_noME11[data_type])>hit_thresh)/len(pg_ncsc_noME11[data_type]))
        eff_ncsc[data_type]['noCluster']['pg_noME112'] = np.append(eff_ncsc[data_type]['noCluster']['pg_noME112'], np.count_nonzero(np.array(pg_ncsc_noME112[data_type])>hit_thresh)/len(pg_ncsc_noME112[data_type]))
        eff_ncsc[data_type]['noCluster']['pg_noME1123'] = np.append(eff_ncsc[data_type]['noCluster']['pg_noME1123'], np.count_nonzero(np.array(pg_ncsc_noME1123[data_type])>hit_thresh)/len(pg_ncsc_noME1123[data_type]))

        
#         Overlap
        eff_nrpc[data_type]['noCluster']['rpc_inclusive'] = np.append(eff_nrpc[data_type]['noCluster']['rpc_inclusive'], np.count_nonzero(np.array(nRpc[data_type])>hit_thresh)/len(nRpc[data_type]))

    # Rejection power calculation
    rejection_power[data_type] = {}
    rejection_power[data_type]['noCluster'] = {}
#     rejection_power[data_type]['noCluster_ratioCut'] = {}
    
    # No clustering
    rejection_power[data_type]['noCluster']['csc_inclusive'] = np.reciprocal(np.trim_zeros(eff_ncsc[data_type]['noCluster']['csc_inclusive']))
    rejection_power[data_type]['noCluster']['csc_noME11'] = np.reciprocal(np.trim_zeros(eff_ncsc[data_type]['noCluster']['csc_noME11']))
    rejection_power[data_type]['noCluster']['csc_noME112'] = np.reciprocal(np.trim_zeros(eff_ncsc[data_type]['noCluster']['csc_noME112']))
    rejection_power[data_type]['noCluster']['csc_noME1123'] = np.reciprocal(np.trim_zeros(eff_ncsc[data_type]['noCluster']['csc_noME1123']))
    
    rejection_power[data_type]['noCluster']['pg_inclusive'] = np.reciprocal(np.trim_zeros(eff_ncsc[data_type]['noCluster']['pg_inclusive']))
    rejection_power[data_type]['noCluster']['pg_noME11'] = np.reciprocal(np.trim_zeros(eff_ncsc[data_type]['noCluster']['pg_noME11']))    
    rejection_power[data_type]['noCluster']['pg_noME112'] = np.reciprocal(np.trim_zeros(eff_ncsc[data_type]['noCluster']['pg_noME112']))
    rejection_power[data_type]['noCluster']['pg_noME1123'] = np.reciprocal(np.trim_zeros(eff_ncsc[data_type]['noCluster']['pg_noME1123']))

    rejection_power[data_type]['noCluster']['rpc_inclusive'] = np.reciprocal(np.trim_zeros(eff_nrpc[data_type]['noCluster']['rpc_inclusive']))

    For the high LLP mass, low ctau sample (m975ct1m) , none of the events pass the barrel cut and thus I hard-coded a 
    way to avoid divide-by-zero errors later on when plotting ROC curves by simpling not recording any barrel datapoints
    for this sample.
    if data_type != 'm975ct1m':
        
        for hit_thresh in range(0, 200):
            ndt_var_x = np.append(ndt_var_x, hit_thresh)
            
            # No-Clustering ROCs
            # Barrel
            eff_ndt[data_type]['noCluster']['dt_inclusive'] = np.append(eff_ndt[data_type]['noCluster']['dt_inclusive'], np.count_nonzero(np.array(nDt[data_type])>hit_thresh)/len(nDt[data_type]))
            eff_ndt[data_type]['noCluster']['dt_noMB1'] = np.append(eff_ndt[data_type]['noCluster']['dt_noMB1'], np.count_nonzero(np.array(nDt_removal[data_type]['MB_1'])>hit_thresh)/len(nDt_removal[data_type]['MB_1']))
            eff_ndt[data_type]['noCluster']['dt_noMB12'] = np.append(eff_ndt[data_type]['noCluster']['dt_noMB12'], np.count_nonzero(np.array(nDt_removal[data_type]['MB_12'])>hit_thresh)/len(nDt_removal[data_type]['MB_12']))
            
#             eff_ndt[data_type]['noCluster_ratioCut']['dt_inclusive'] = np.append(eff_ndt[data_type]['noCluster_ratioCut']['dt_inclusive'], np.count_nonzero(np.array(nDt_ratio_cut[data_type]['dt_inclusive'])>hit_thresh)/len(nDt_ratio_cut[data_type]['dt_inclusive']))
#             eff_ndt[data_type]['noCluster_ratioCut']['dt_noMB1'] = np.append(eff_ndt[data_type]['noCluster_ratioCut']['dt_noMB1'], np.count_nonzero(np.array(nDt_ratio_cut[data_type]['no_MB1'])>hit_thresh)/len(nDt_ratio_cut[data_type]['no_MB1']))
            
            
        # No clustering
        rejection_power[data_type]['noCluster']['dt_inclusive'] = np.reciprocal(np.trim_zeros(eff_ndt[data_type]['noCluster']['dt_inclusive']))
        rejection_power[data_type]['noCluster']['dt_noMB1'] = np.reciprocal(np.trim_zeros(eff_ndt[data_type]['noCluster']['dt_noMB1']))
        rejection_power[data_type]['noCluster']['dt_noMB12'] = np.reciprocal(np.trim_zeros(eff_ndt[data_type]['noCluster']['dt_noMB12']))
        
#         rejection_power[data_type]['noCluster_ratioCut']['dt_inclusive'] = np.reciprocal(np.trim_zeros(eff_ndt[data_type]['noCluster_ratioCut']['dt_inclusive']))  
#         rejection_power[data_type]['noCluster_ratioCut']['dt_noMB1'] = np.reciprocal(np.trim_zeros(eff_ndt[data_type]['noCluster_ratioCut']['dt_noMB1']))  

## Event Lookup Framework

In [ ]:
# # Selection of Events for Event Display Purposes (Test area)
# # print(len(eventNum_barrel['m50ct10m']))
# # print(len(nDt_avg_ratio['m50ct10m'][0]))
# index_ev = np.where(np.array(nDt['m50ct10m'])==0)
# print('Lumi ' + 'Event ' + 'nCsc ' + 'gLLP_r')
# print(np.dstack((lumiNum_barrel['m50ct10m'][index_ev].astype(int), eventNum_barrel['m50ct10m'][index_ev].astype(int), gLLPr['m50ct10m'][index_ev])))

# Endcap Offline Study

## *Endcap Inclusive N<sub>CSC</sub> Histogram*

In [ ]:
# Creates a 1D histogram of the number of CSC hits for the m50ct1m signal MC, QCD MC, and ZeroBias data 
# with no hit removal

c = rt.TCanvas('c','c', 800, 600)
h = {}
h['nCsc_m50ct1m'] = create_TH1D(nCsc['m50ct1m'], axis_title=['nCsc_m50ct1m', 'Events'], name='nCsc_m50ct1m', binning=[150,0,500])
h['nCsc_m50ct1m'].SetLineColor(4)

h['nCsc_qcd'] = create_TH1D(nCsc['qcd'], axis_title=['nCsc_qcd', 'Events'], name='nCsc_qcd', binning=[150,0,500])
h['nCsc_qcd'].SetLineColor(2)
h['nCsc_qcd'].SetLineStyle(2)


h['nCsc_zeroBias'] = create_TH1D(nCsc['zeroBias'], axis_title=['nCsc_zeroBias', 'Events'], name='nCsc_zeroBias', binning=[150,0,500])
h['nCsc_zeroBias'].SetLineColor(2)
h['nCsc_zeroBias'].SetLineStyle(1)

c.SetLogy()

h['nCsc_m50ct1m'].Scale(1.0/h['nCsc_m50ct1m'].Integral())
h['nCsc_qcd'].Scale(1.0/h['nCsc_qcd'].Integral())
h['nCsc_zeroBias'].Scale(1.0/h['nCsc_zeroBias'].Integral())


h['nCsc_m50ct1m'].SetLineWidth(2)
h['nCsc_qcd'].SetLineWidth(2)
h['nCsc_zeroBias'].SetLineWidth(2)

h['nCsc_m50ct1m'].GetXaxis().SetRangeUser(0,200)
h['nCsc_qcd'].GetXaxis().SetRangeUser(0,200)
h['nCsc_zeroBias'].GetXaxis().SetRangeUser(0,200)


h['nCsc_qcd'].SetStats(0)
h['nCsc_qcd'].SetTitle("CSC Inclusive")
h['nCsc_qcd'].SetXTitle("N_{csc}")


h['nCsc_qcd'].Draw('histo')
h['nCsc_m50ct1m'].Draw('histo+same')
h['nCsc_zeroBias'].Draw('histo+same')

legend = rt.TLegend(0.50,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
#legend-.SetFillStyle(0);
legend.AddEntry( h['nCsc_m50ct1m'], "mX=50 GeV, c#tau=1m (MC)" , "L");
legend.AddEntry( h['nCsc_qcd'], "QCD (MC)" , "L");
legend.AddEntry( h['nCsc_zeroBias'], "Zero Bias" , "L");

legend.Draw();

c.Draw()

c.SaveAs("ncsc_inclusive_mc_zeroBias.pdf")
c.SaveAs("ncsc_inclusive_mc_zeroBias.C")

## *N<sub>CSC</sub> Histogram with ME 1/1 Hits Removed*

In [ ]:
# Creates a 1D histogram of the number of CSC hits for the m50ct1m signal MC, QCD MC, and ZeroBias data
# after removing hits in ME 1/1

c2 = rt.TCanvas('c2','c2', 800, 600)

h['nCsc_m50ct1m_noME11'] = create_TH1D(nCsc_removal['m50ct1m']['ME_11'], axis_title=['nCsc_m50ct1m_noME11', 'Events'], name='nCsc_m50ct1m_noME11', binning=[150,0,500])
h['nCsc_m50ct1m_noME11'].SetLineColor(4)

h['nCsc_qcd_noME11'] = create_TH1D(nCsc_removal['qcd']['ME_11'], axis_title=['nCsc_qcd_noME11', 'Events'], name='nCsc_qcd_noME11', binning=[150,0,500])
h['nCsc_qcd_noME11'].SetLineColor(2)
h['nCsc_qcd_noME11'].SetLineStyle(2)


h['nCsc_zeroBias_noME11'] = create_TH1D(nCsc_removal['zeroBias']['ME_11'], axis_title=['nCsc_zeroBias_noME11', 'Events'], name='nCsc_zeroBias_noME11', binning=[150,0,500])
h['nCsc_zeroBias_noME11'].SetLineColor(2)
h['nCsc_zeroBias_noME11'].SetLineStyle(1)

c2.SetLogy()

h['nCsc_m50ct1m_noME11'].Scale(1.0/h['nCsc_m50ct1m_noME11'].Integral())
h['nCsc_qcd_noME11'].Scale(1.0/h['nCsc_qcd_noME11'].Integral())
h['nCsc_zeroBias_noME11'].Scale(1.0/h['nCsc_zeroBias_noME11'].Integral())


h['nCsc_m50ct1m_noME11'].SetLineWidth(2)
h['nCsc_qcd_noME11'].SetLineWidth(2)
h['nCsc_zeroBias_noME11'].SetLineWidth(2)

h['nCsc_m50ct1m_noME11'].GetXaxis().SetRangeUser(0,200)
h['nCsc_qcd_noME11'].GetXaxis().SetRangeUser(0,200)
h['nCsc_zeroBias_noME11'].GetXaxis().SetRangeUser(0,200)


h['nCsc_qcd_noME11'].SetStats(0)
h['nCsc_qcd_noME11'].SetTitle("ME 1/1 Removed")
h['nCsc_qcd_noME11'].SetXTitle("N_{csc}")


h['nCsc_qcd_noME11'].Draw('histo')
h['nCsc_m50ct1m_noME11'].Draw('histo+same')
h['nCsc_zeroBias_noME11'].Draw('histo+same')

legend = rt.TLegend(0.50,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
#legend-.SetFillStyle(0);
legend.AddEntry( h['nCsc_m50ct1m_noME11'], "mX=50 GeV, c#tau=1m (MC)" , "L");
legend.AddEntry( h['nCsc_qcd_noME11'], "QCD (MC)" , "L");
legend.AddEntry( h['nCsc_zeroBias_noME11'], "Zero Bias" , "L");

legend.Draw();

c2.Draw()

c2.SaveAs("ncsc_noME11_mc_zeroBias.pdf")
c2.SaveAs("ncsc_noME11_mc_zeroBias.C")

## *N<sub>CSC</sub> Threshold ROC Curves*

In [ ]:
# Creates ROC curves for the signal efficiency of the m50ct1m signal sample against the ZeroBias samples
# using nCsc thresholds

c3 = rt.TCanvas('c3','c3', 800, 600)

h['roc_curve_noCluster'] = create_TGraph(rejection_power['zeroBias']['noCluster']['csc_inclusive'], eff_ncsc['m50ct1m']['noCluster']['csc_inclusive'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_curve_clean_noCluster'] = create_TGraph(rejection_power['zeroBias']['noCluster']['csc_noME11'], eff_ncsc['m50ct1m']['noCluster']['csc_noME11'], axis_title=['#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_curve_res_noCluster'] = create_TGraph(rejection_power['zeroBias']['noCluster']['csc_noME1123'], eff_ncsc['m50ct1m']['noCluster']['csc_noME1123'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_curve_med_noCluster'] = create_TGraph(rejection_power['zeroBias']['noCluster']['csc_noME112'], eff_ncsc['m50ct1m']['noCluster']['csc_noME112'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])

h['roc_curve_pg_inc'] = create_TGraph(rejection_power['zeroBias']['noCluster']['pg_inclusive'], eff_ncsc['m50ct1m']['noCluster']['pg_inclusive'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_curve_pg_no11'] = create_TGraph(rejection_power['zeroBias']['noCluster']['pg_noME11'], eff_ncsc['m50ct1m']['noCluster']['pg_noME11'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_curve_pg_no112'] = create_TGraph(rejection_power['zeroBias']['noCluster']['pg_noME112'], eff_ncsc['m50ct1m']['noCluster']['pg_noME112'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_curve_pg_no1123'] = create_TGraph(rejection_power['zeroBias']['noCluster']['pg_noME1123'], eff_ncsc['m50ct1m']['noCluster']['pg_noME1123'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])


c3.SetLogx()

h['roc_curve_noCluster'].SetTitle('ZeroBias')


h['roc_curve_noCluster'].SetLineWidth(2)
h['roc_curve_clean_noCluster'].SetLineWidth(2)
h['roc_curve_res_noCluster'].SetLineWidth(2)
h['roc_curve_med_noCluster'].SetLineWidth(2)

h['roc_curve_pg_inc'].SetLineWidth(2)
h['roc_curve_pg_no11'].SetLineWidth(2)
h['roc_curve_pg_no112'].SetLineWidth(2)
h['roc_curve_pg_no1123'].SetLineWidth(2)


h['roc_curve_noCluster'].SetLineColor(1)
h['roc_curve_noCluster'].SetLineStyle(2)

h['roc_curve_clean_noCluster'].SetLineColor(2)
h['roc_curve_clean_noCluster'].SetLineStyle(2)
                                          
h['roc_curve_res_noCluster'].SetLineColor(3)
h['roc_curve_res_noCluster'].SetLineStyle(2)

h['roc_curve_med_noCluster'].SetLineColor(4)
h['roc_curve_med_noCluster'].SetLineStyle(2)

h['roc_curve_pg_inc'].SetLineColor(1)
h['roc_curve_pg_no11'].SetLineColor(2)
h['roc_curve_pg_no112'].SetLineColor(3)
h['roc_curve_pg_no1123'].SetLineColor(4)

h['roc_curve_noCluster'].GetXaxis().SetLimits(10, 10000000)
h['roc_curve_noCluster'].GetYaxis().SetRangeUser(0,1)

h['roc_curve_noCluster'].Draw()
h['roc_curve_clean_noCluster'].Draw('same')

h['roc_curve_res_noCluster'].Draw('same')
h['roc_curve_med_noCluster'].Draw('same')

h['roc_curve_pg_inc'].Draw('same')
h['roc_curve_pg_no11'].Draw('same')
h['roc_curve_pg_no112'].Draw('same')
h['roc_curve_pg_no1123'].Draw('same')

legend = rt.TLegend(0.50,0.70,0.5,0.7);
legend.SetTextSize(0.03);
legend.SetBorderSize(0);
legend.SetFillStyle(0);

legend.AddEntry( h['roc_curve_noCluster'], "Reco Inclusive", 'l');
legend.AddEntry( h['roc_curve_clean_noCluster'], "Reco No 1/1", 'l');
legend.AddEntry( h['roc_curve_med_noCluster'], "Reco No 1/1+1/2", 'l');
legend.AddEntry( h['roc_curve_res_noCluster'], "Reco No 1/1+1/2+1/3", 'l');

legend.AddEntry( h['roc_curve_pg_inc'], "L1 Inclusive", 'l');
legend.AddEntry( h['roc_curve_pg_no11'], "L1 No 1/1", 'l');
legend.AddEntry( h['roc_curve_pg_no112'], "L1 No 1/1+1/2", 'l');
legend.AddEntry( h['roc_curve_pg_no1123'], "L1 No 1/1+1/2+1/3", 'l');


legend.Draw();

c3.Draw()

# Barrel Offline Studies

## *Barrel Inclusive N<sub>DT</sub> Histogram*

In [ ]:
# Creates a 1D histogram of the number of DT hits for the m50ct1m signal MC, QCD MC, and ZeroBias data
# with no hit removal

c4 = rt.TCanvas('c4','c4', 800, 600)
h['nDt_signal'] = create_TH1D(nDt['m50ct1m'], axis_title=['nDt_signal', 'Events'], name='nDt_signal', binning=[150,0,500])
h['nDt_signal'].SetLineColor(4)

h['nDt_qcd'] = create_TH1D(nDt['qcd'], axis_title=['nDt_qcd', 'Events'], name='nDt_qcd', binning=[150,0,500])
h['nDt_qcd'].SetLineColor(2)
h['nDt_qcd'].SetLineStyle(2)


h['nDt_zeroBias'] = create_TH1D(nDt['zeroBias'], axis_title=['nDt_zeroBias', 'Events'], name='nDt_zeroBias', binning=[150,0,500])
h['nDt_zeroBias'].SetLineColor(2)
h['nDt_zeroBias'].SetLineStyle(1)

c4.SetLogy()

h['nDt_signal'].Scale(1.0/h['nDt_signal'].Integral())
h['nDt_qcd'].Scale(1.0/h['nDt_qcd'].Integral())
h['nDt_zeroBias'].Scale(1.0/h['nDt_zeroBias'].Integral())


h['nDt_signal'].SetLineWidth(2)
h['nDt_qcd'].SetLineWidth(2)
h['nDt_zeroBias'].SetLineWidth(2)

h['nDt_signal'].GetXaxis().SetRangeUser(0,200)
h['nDt_qcd'].GetXaxis().SetRangeUser(0,200)
h['nDt_zeroBias'].GetXaxis().SetRangeUser(0,200)


h['nDt_qcd'].SetStats(0)
h['nDt_qcd'].SetTitle("DT Inclusive")
h['nDt_qcd'].SetXTitle("N_{DT}")


h['nDt_qcd'].Draw('histo')
h['nDt_signal'].Draw('histo+same')
h['nDt_zeroBias'].Draw('histo+same')

legend = rt.TLegend(0.49,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
legend.AddEntry( h['nDt_signal'], "mX=50 GeV, c#tau=1m (MC)" , "L");
legend.AddEntry( h['nDt_qcd'], "QCD (MC)" , "L");
legend.AddEntry( h['nDt_zeroBias'], "Zero Bias" , "L");

legend.Draw();

c4.Draw()

## *N<sub>DT</sub> Histogram with MB 1 Hits Removed*

In [ ]:
# Creates a 1D histogram of the number of DT hits for the m50ct1m signal MC, QCD MC, and ZeroBias data
# after removing hits in MB 1

c5 = rt.TCanvas('c5','c5', 800, 600)

h['nDt_signal_noMB1'] = create_TH1D(nDt_removal['m50ct1m']['MB_1'], axis_title=['nDt_signal_noMB1', 'Events'], name='nDt_signal_noMB1', binning=[150,0,500])
h['nDt_signal_noMB1'].SetLineColor(4)

h['nDt_qcd_noMB1'] = create_TH1D(nDt_removal['qcd']['MB_1'], axis_title=['nDt_qcd_noMB1', 'Events'], name='nDt_qcd_noMB1', binning=[150,0,500])
h['nDt_qcd_noMB1'].SetLineColor(2)
h['nDt_qcd_noMB1'].SetLineStyle(2)


h['nDt_zeroBias_noMB1'] = create_TH1D(nDt_removal['zeroBias']['MB_1'], axis_title=['nDt_zeroBias_noMB1', 'Events'], name='nDt_zeroBias_noMB1', binning=[150,0,500])
h['nDt_zeroBias_noMB1'].SetLineColor(2)
h['nDt_zeroBias_noMB1'].SetLineStyle(1)

c5.SetLogy()

h['nDt_signal_noMB1'].Scale(1.0/h['nDt_signal_noMB1'].Integral())
h['nDt_qcd_noMB1'].Scale(1.0/h['nDt_qcd_noMB1'].Integral())
h['nDt_zeroBias_noMB1'].Scale(1.0/h['nDt_zeroBias_noMB1'].Integral())


h['nDt_signal_noMB1'].SetLineWidth(2)
h['nDt_qcd_noMB1'].SetLineWidth(2)
h['nDt_zeroBias_noMB1'].SetLineWidth(2)

h['nDt_qcd_noMB1'].GetXaxis().SetRangeUser(0,200)
h['nDt_qcd_noMB1'].SetStats(0)
h['nDt_qcd_noMB1'].SetTitle("MB 1 Removed")
h['nDt_qcd_noMB1'].SetXTitle("N_{DT}")


h['nDt_qcd_noMB1'].Draw('histo')
h['nDt_signal_noMB1'].Draw('histo+same')
h['nDt_zeroBias_noMB1'].Draw('histo+same')

legend = rt.TLegend(0.48,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
#legend-.SetFillStyle(0);
legend.AddEntry( h['nDt_signal_noMB1'], "mX=50 GeV, c#tau=1m (MC)" , "L");
legend.AddEntry( h['nDt_qcd_noMB1'], "QCD (MC)" , "L");
legend.AddEntry( h['nDt_zeroBias_noMB1'], "Zero Bias" , "L");

legend.Draw();

c5.Draw()

In [ ]:
# Creates a 1D histogram of the number of DT hits for the m50ct1m signal MC, QCD MC, and ZeroBias data
# after removing hits in MB 1

c5_5 = rt.TCanvas('c5_5','c5_5', 800, 600)

h['nDt_signal_MB1'] = create_TH1D(nDt_station['m50ct10m']['MB_1'], axis_title=['nDt_signal_MB1', 'Events'], name='nDt_signal_MB1', binning=[50,0,200])
h['nDt_signal_MB1'].SetLineColor(1)
h['nDt_signal_MB1'].SetLineWidth(2)

h['nDt_signal_MB2'] = create_TH1D(nDt_station['m50ct10m']['MB_2'], axis_title=['nDt_signal_MB2', 'Events'], name='nDt_signal_MB2', binning=[50,0,200])
h['nDt_signal_MB2'].SetLineColor(2)
h['nDt_signal_MB2'].SetLineWidth(2)

h['nDt_signal_MB3'] = create_TH1D(nDt_station['m50ct10m']['MB_3'], axis_title=['nDt_signal_MB3', 'Events'], name='nDt_signal_MB3', binning=[50,0,200])
h['nDt_signal_MB3'].SetLineColor(3)
h['nDt_signal_MB3'].SetLineWidth(2)

h['nDt_signal_MB4'] = create_TH1D(nDt_station['m50ct10m']['MB_4'], axis_title=['nDt_signal_MB4', 'Events'], name='nDt_signal_MB4', binning=[50,0,200])
h['nDt_signal_MB4'].SetLineColor(4)
h['nDt_signal_MB4'].SetLineWidth(2)

c5_5.SetLogy()

h['nDt_signal_MB1'].Scale(1.0/h['nDt_signal_MB1'].Integral())
h['nDt_signal_MB2'].Scale(1.0/h['nDt_signal_MB2'].Integral())
h['nDt_signal_MB3'].Scale(1.0/h['nDt_signal_MB3'].Integral())
h['nDt_signal_MB4'].Scale(1.0/h['nDt_signal_MB4'].Integral())


h['nDt_signal_MB1'].GetXaxis().SetRangeUser(0,120)
h['nDt_signal_MB1'].SetStats(0)
h['nDt_signal_MB1'].SetTitle("N_{DT} per station: mX=50 GeV, c#tau=10m (MC)")
h['nDt_signal_MB1'].SetXTitle("N_{DT}")


h['nDt_signal_MB1'].Draw('histo')
h['nDt_signal_MB2'].Draw('histo+same')
h['nDt_signal_MB3'].Draw('histo+same')
h['nDt_signal_MB4'].Draw('histo+same')

legend = rt.TLegend(0.48,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
#legend-.SetFillStyle(0);
legend.AddEntry( h['nDt_signal_MB1'], "MB 1" , "L");
legend.AddEntry( h['nDt_signal_MB2'], "MB 2" , "L");
legend.AddEntry( h['nDt_signal_MB3'], "MB 3" , "L");
legend.AddEntry( h['nDt_signal_MB4'], "MB 4" , "L");

legend.Draw();

c5_5.Draw()

## *N<sub>DT</sub> Histogram for MB 2 Hits Only*

In [ ]:
# Creates a 1D histogram of the number of DT hits for the m50ct1m signal MC, QCD MC, and ZeroBias data in MB 2

c5_comp = rt.TCanvas('c5_comp','c5_comp', 800, 600)

h['nDt_m50ct1m_MB1'] = create_TH1D(nDt_station['m50ct1m']['MB_2'], axis_title=['nDt_m50ct1m_MB1', 'Events'], name='nDt_m50ct1m_MB1', binning=[50,0,200])
h['nDt_m50ct1m_MB1'].SetLineColor(1)
h['nDt_m50ct1m_MB1'].SetLineWidth(2)

h['nDt_qcd_MB1'] = create_TH1D(nDt_station['qcd']['MB_2'], axis_title=['nDt_qcd_MB1', 'Events'], name='nDt_qcd_MB1', binning=[50,0,200])
h['nDt_qcd_MB1'].SetLineColor(2)
h['nDt_qcd_MB1'].SetLineWidth(2)

h['nDt_zeroBias_MB1'] = create_TH1D(nDt_station['zeroBias']['MB_2'], axis_title=['nDt_zeroBias_MB1', 'Events'], name='nDt_zeroBias_MB1', binning=[50,0,200])
h['nDt_zeroBias_MB1'].SetLineColor(3)
h['nDt_zeroBias_MB1'].SetLineWidth(2)

h['nDt_m50ct10m_MB1'] = create_TH1D(nDt_station['m50ct10m']['MB_2'], axis_title=['nDt_m50ct10m_MB1', 'Events'], name='nDt_m50ct10m_MB1', binning=[50,0,200])
h['nDt_m50ct10m_MB1'].SetLineColor(4)
h['nDt_m50ct10m_MB1'].SetLineWidth(2)

c5_comp.SetLogy()

h['nDt_m50ct1m_MB1'].Scale(1.0/h['nDt_m50ct1m_MB1'].Integral())
h['nDt_qcd_MB1'].Scale(1.0/h['nDt_qcd_MB1'].Integral())
h['nDt_zeroBias_MB1'].Scale(1.0/h['nDt_zeroBias_MB1'].Integral())
h['nDt_m50ct10m_MB1'].Scale(1.0/h['nDt_m50ct10m_MB1'].Integral())


h['nDt_m50ct1m_MB1'].GetXaxis().SetRangeUser(0,120)
h['nDt_m50ct1m_MB1'].SetStats(0)
h['nDt_m50ct1m_MB1'].SetTitle("MB 2 N_{DT} per station: mX=50 GeV, c#tau=10m (MC)")
h['nDt_m50ct1m_MB1'].SetXTitle("N_{DT}")


h['nDt_m50ct1m_MB1'].Draw('histo')
h['nDt_qcd_MB1'].Draw('histo+same')
h['nDt_zeroBias_MB1'].Draw('histo+same')
h['nDt_m50ct10m_MB1'].Draw('histo+same')

legend = rt.TLegend(0.48,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
#legend-.SetFillStyle(0);
legend.AddEntry( h['nDt_m50ct1m_MB1'], "mX=50 GeV, c#tau=1m (MC)" , "L");
legend.AddEntry( h['nDt_qcd_MB1'], "QCD" , "L");
legend.AddEntry( h['nDt_zeroBias_MB1'], "ZeroBias" , "L");
legend.AddEntry( h['nDt_m50ct10m_MB1'], "mX=50 GeV, c#tau=10m (MC)" , "L");

legend.Draw();

c5_comp.Draw()

## *Average Fraction of Hits per DT station*

In [ ]:
# Creates a 1D histogram of the average fraction of DT hits contributed by each barrel station

c5_6 = rt.TCanvas('c5_6','c5_6', 800, 600)

h['nDt_signal_ratio'] = create_TH1D(nDt_avg_ratio['m50ct10m'][0], axis_title=['nDt_signal_ratio', 'Events'], name='nDt_signal_ratio', binning=[50,0,5])
h['nDt_signal_ratio'].SetLineColor(4)
h['nDt_signal_ratio'].SetLineWidth(2)

h['nDt_qcd_ratio'] = create_TH1D(nDt_avg_ratio['qcd'][0], axis_title=['nDt_qcd_ratio', 'Events'], name='nDt_qcd_ratio', binning=[50,0,5])
h['nDt_qcd_ratio'].SetLineColor(2)
h['nDt_qcd_ratio'].SetLineStyle(2)
h['nDt_qcd_ratio'].SetLineWidth(2)

h['nDt_zeroBias_ratio'] = create_TH1D(nDt_avg_ratio['zeroBias'][0], axis_title=['nDt_zeroBias_ratio', 'Events'], name='nDt_zeroBias_ratio', binning=[50,0,5])
h['nDt_zeroBias_ratio'].SetLineColor(2)
h['nDt_zeroBias_ratio'].SetLineStyle(1)
h['nDt_zeroBias_ratio'].SetLineWidth(1)


c5_6.SetLogy()

h['nDt_signal_ratio'].Scale(1.0/h['nDt_signal_ratio'].Integral())
h['nDt_qcd_ratio'].Scale(1.0/h['nDt_qcd_ratio'].Integral())
h['nDt_zeroBias_ratio'].Scale(1.0/h['nDt_zeroBias_ratio'].Integral())

h['nDt_signal_ratio'].GetXaxis().SetRangeUser(0,1.5)
h['nDt_signal_ratio'].SetStats(0)
h['nDt_signal_ratio'].SetTitle("")
h['nDt_signal_ratio'].SetXTitle("N_{DT} Fraction")


h['nDt_signal_ratio'].Draw('histo')
h['nDt_qcd_ratio'].Draw('histo + same')
h['nDt_zeroBias_ratio'].Draw('histo + same')


legend = rt.TLegend(0.5,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
legend.SetFillStyle(0);
legend.AddEntry( h['nDt_signal_ratio'], "mX=50 GeV, c#tau=10m (MC)" , "L");
legend.AddEntry( h['nDt_qcd_ratio'], "QCD (MC)" , "L");
legend.AddEntry( h['nDt_zeroBias_ratio'], "Zero Bias" , "L");

legend.Draw();

c5_6.Draw()

## *N<sub>DT</sub> Threshold ROC Curves*

In [ ]:
# Creates ROC curves for the signal efficiency of the m50ct1m signal sample against the ZeroBias/QCD samples
# using nDt thresholds

c7 = rt.TCanvas('c7','c7', 800, 600)

h['roc_inclusive_zeroBias'] = create_TGraph(rejection_power['zeroBias']['noCluster']['dt_inclusive'], eff_ndt['m50ct10m']['noCluster']['dt_inclusive'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_no_mb1_zeroBias'] = create_TGraph(rejection_power['zeroBias']['noCluster']['dt_noMB1'], eff_ndt['m50ct10m']['noCluster']['dt_noMB1'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_no_mb12_zeroBias'] = create_TGraph(rejection_power['zeroBias']['noCluster']['dt_noMB12'], eff_ndt['m50ct10m']['noCluster']['dt_noMB12'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
# h['roc_no_mb1_ratioCut_zeroBias'] = create_TGraph(rejection_power['zeroBias']['noCluster_ratioCut']['dt_noMB1'], eff_ndt['m50ct10m']['noCluster_ratioCut']['dt_noMB1'], axis_title=['#epsilon_{bkg}', '#epsilon_{signal}'])


h['roc_inclusive_qcd'] = create_TGraph(rejection_power['qcd']['noCluster']['dt_inclusive'], eff_ndt['m50ct10m']['noCluster']['dt_inclusive'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_no_mb1_qcd'] = create_TGraph(rejection_power['qcd']['noCluster']['dt_noMB1'], eff_ndt['m50ct10m']['noCluster']['dt_noMB1'], axis_title=['#epsilon_{bkg}', '#epsilon_{signal}'])
h['roc_no_mb12_qcd'] = create_TGraph(rejection_power['qcd']['noCluster']['dt_noMB12'], eff_ndt['m50ct10m']['noCluster']['dt_noMB12'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])
# h['roc_no_mb1_ratioCut_qcd'] = create_TGraph(rejection_power['qcd']['noCluster_ratioCut']['dt_noMB1'], eff_ndt['m50ct10m']['noCluster_ratioCut']['dt_noMB1'], axis_title=['#epsilon_{bkg}', '#epsilon_{signal}'])

c7.SetLogx()
# c7.SetLogy()

h['roc_inclusive_zeroBias'].SetTitle('N_{DT}: mX=50 GeV, c#tau=10m (MC)')

h['roc_inclusive_zeroBias'].SetLineWidth(2)
h['roc_no_mb1_zeroBias'].SetLineWidth(2)
# h['roc_no_mb1_ratioCut_zeroBias'].SetLineWidth(2)
h['roc_no_mb12_zeroBias'].SetLineWidth(2)

h['roc_inclusive_qcd'].SetLineWidth(2)
h['roc_no_mb1_qcd'].SetLineWidth(2)
# h['roc_no_mb1_ratioCut_qcd'].SetLineWidth(2)
h['roc_no_mb12_qcd'].SetLineWidth(2)

### ZeroBias #6

h['roc_inclusive_zeroBias'].SetLineColor(1)
h['roc_inclusive_zeroBias'].SetLineStyle(2)


h['roc_no_mb1_zeroBias'].SetLineColor(2)
h['roc_no_mb1_zeroBias'].SetLineStyle(2)

h['roc_no_mb12_zeroBias'].SetLineColor(4)
h['roc_no_mb12_zeroBias'].SetLineStyle(2)

# h['roc_no_mb1_ratioCut_zeroBias'].SetLineColor(4)
# h['roc_no_mb1_ratioCut_zeroBias'].SetLineStyle(2)


### QCD

# h['roc_inclusive_qcd'].SetLineColor(1)
# h['roc_inclusive_qcd'].SetLineStyle(1)

h['roc_no_mb1_qcd'].SetLineColor(2)
h['roc_no_mb1_qcd'].SetLineStyle(1)

h['roc_no_mb12_qcd'].SetLineColor(4)
h['roc_no_mb12_qcd'].SetLineStyle(1)

# h['roc_no_mb1_ratioCut_qcd'].SetLineColor(4)
# h['roc_no_mb1_ratioCut_qcd'].SetLineStyle(1)

####

h['roc_no_mb1_zeroBias'].GetXaxis().SetLimits(10, 10000000)
h['roc_no_mb1_zeroBias'].GetYaxis().SetRangeUser(0,1)



h['roc_inclusive_zeroBias'].Draw()
h['roc_no_mb1_zeroBias'].Draw('same')
h['roc_no_mb12_zeroBias'].Draw('same')
# h['roc_no_mb1_ratioCut_zeroBias'].Draw('same')

h['roc_inclusive_qcd'].Draw('same')
h['roc_no_mb1_qcd'].Draw('same')
h['roc_no_mb12_qcd'].Draw('same')
# h['roc_no_mb1_ratioCut_qcd'].Draw('same')


legend = rt.TLegend(0.60,0.60,0.6,0.6);
legend.SetTextSize(0.03);
legend.SetBorderSize(0);
legend.SetFillStyle(0);

legend.AddEntry( h['roc_inclusive_zeroBias'], "ZeroBias", 'l');
legend.AddEntry( h['roc_no_mb1_zeroBias'], "ZeroBias w/o 1/1", 'l');
legend.AddEntry( h['roc_no_mb12_zeroBias'], "ZeroBias w/o 1/1+1/2", 'l');
# legend.AddEntry( h['roc_no_mb1_ratioCut_zeroBias'], "ZeroBias w/o 1/1 + Fraction Cut", 'l');

legend.AddEntry( h['roc_inclusive_qcd'], "QCD", 'l');
legend.AddEntry( h['roc_no_mb1_qcd'], "QCD w/o 1/1", 'l');
legend.AddEntry( h['roc_no_mb12_qcd'], "QCD w/o 1/1+1/2", 'l');
# legend.AddEntry( h['roc_no_mb1_ratioCut_qcd'], "QCD w/o 1/1 + Fraction Cut", 'l');



legend.Draw();

c7.Draw()

# Overlap Offline Studies

## *Overlap Inclusive N<sub>RPC</sub> Histogram*

In [ ]:
# Creates a 1D histogram of the number of RPC hits for the m50ct1m signal MC, QCD MC, and ZeroBias data
# with no hit removal

c6 = rt.TCanvas('c6','c6', 800, 600)
h['nRpc_signal'] = create_TH1D(nRpc['m50ct1m'], axis_title=['nRpc_signal', 'Events'], name='nRpc_signal', binning=[150,0,500])
h['nRpc_signal'].SetLineColor(4)

h['nRpc_qcd'] = create_TH1D(nRpc['qcd'], axis_title=['nRpc_qcd', 'Events'], name='nRpc_qcd', binning=[150,0,500])
h['nRpc_qcd'].SetLineColor(2)
h['nRpc_qcd'].SetLineStyle(2)


h['nRpc_zeroBias'] = create_TH1D(nRpc['zeroBias'], axis_title=['nRpc_zeroBias', 'Events'], name='nRpc_zeroBias', binning=[150,0,500])
h['nRpc_zeroBias'].SetLineColor(2)
h['nRpc_zeroBias'].SetLineStyle(1)

c6.SetLogy()

h['nRpc_signal'].Scale(1.0/h['nRpc_signal'].Integral())
h['nRpc_qcd'].Scale(1.0/h['nRpc_qcd'].Integral())
h['nRpc_zeroBias'].Scale(1.0/h['nRpc_zeroBias'].Integral())


h['nRpc_signal'].SetLineWidth(2)
h['nRpc_qcd'].SetLineWidth(2)
h['nRpc_zeroBias'].SetLineWidth(2)

h['nRpc_signal'].GetXaxis().SetRangeUser(0,200)


h['nRpc_signal'].SetStats(0)
h['nRpc_signal'].SetTitle("RPC Inclusive")
h['nRpc_signal'].SetXTitle("N_{DT}")


h['nRpc_signal'].Draw('histo')
h['nRpc_qcd'].Draw('histo+same')
h['nRpc_zeroBias'].Draw('histo+same')

legend = rt.TLegend(0.50,0.70,0.87,0.87);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
legend.AddEntry( h['nRpc_signal'], "mX=50 GeV, c#tau=1m (MC)" , "L");
legend.AddEntry( h['nRpc_qcd'], "QCD (MC)" , "L");
legend.AddEntry( h['nRpc_zeroBias'], "Zero Bias" , "L");

legend.Draw();

c6.Draw()

## *N<sub>RPC</sub> Threshold ROC Curves*

In [ ]:
# Creates ROC curves for the signal efficiency of the m50ct1m signal sample against the ZeroBias/QCD samples
# using nRpc thresholds

c8 = rt.TCanvas('c8','c8', 800, 600)

h['roc_inclusive_zeroBias_rpc'] = create_TGraph(rejection_power['zeroBias']['noCluster']['rpc_inclusive'], eff_nrpc['m50ct1m']['noCluster']['rpc_inclusive'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])

h['roc_inclusive_qcd_rpc'] = create_TGraph(rejection_power['qcd']['noCluster']['rpc_inclusive'], eff_nrpc['m50ct1m']['noCluster']['rpc_inclusive'], axis_title=['1/#epsilon_{bkg}', '#epsilon_{signal}'])

c8.SetLogx()

h['roc_inclusive_zeroBias_rpc'].SetTitle('N_{RPC}: mX=50 GeV, c#tau=1m (MC)')

h['roc_inclusive_zeroBias_rpc'].SetLineWidth(2)
h['roc_inclusive_qcd_rpc'].SetLineWidth(2)

h['roc_inclusive_zeroBias_rpc'].SetLineColor(2)
h['roc_inclusive_qcd_rpc'].SetLineColor(4)

h['roc_inclusive_zeroBias_rpc'].GetXaxis().SetLimits(10, 10000000)
h['roc_inclusive_zeroBias_rpc'].GetYaxis().SetRangeUser(0,1)



h['roc_inclusive_zeroBias_rpc'].Draw()
h['roc_inclusive_qcd_rpc'].Draw('same')


legend = rt.TLegend(0.70,0.70,0.7,0.7);
legend.SetTextSize(0.04);
legend.SetBorderSize(0);
legend.SetFillStyle(0);

legend.AddEntry( h['roc_inclusive_zeroBias_rpc'], "ZeroBias", 'l');
legend.AddEntry( h['roc_inclusive_qcd_rpc'], "QCD", 'l');

legend.Draw();

c8.Draw()